In [1]:
import pandas as pd

In [2]:
pd.read_csv('../data/allinone.csv')

,Unnamed: 0,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,54865,3,2,0,12,0,6,6,6.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,1,55054,109,1,1,6,6,6,6,6.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,2,55055,52,1,1,6,6,6,6,6.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,3,46236,34,1,1,6,6,6,6,6.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,4,54863,3,2,0,12,0,6,6,6.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3072054,3072054,53,32215,4,2,112,152,28,28,28.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3072055,3072055,53,324,2,2,84,362,42,42,42.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3072056,3072056,58030,82,2,1,31,6,31,0,15.5,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3072057,3072057,53,1048635,6,2,192,256,32,32,32.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
